In [1]:
ROOT_DIR = r"C:\Users\rajes\OneDrive\Desktop\DAYUMMMMMMMMM\CNN"

In [1]:
import ultralytics
from ultralytics import YOLO

model = YOLO(r'C:\Users\rajes\OneDrive\Desktop\DAYUMMMMMMMMM\CNN\runs\detect\train44\weights\best.pt')

results = model(r'C:\Users\rajes\OneDrive\Desktop\DAYUMMMMMMMMM\CNN\images (1).jpg')

for result in results:
    boxes = result.boxes.xyxy.cpu().numpy()  
    scores = result.boxes.conf.cpu().numpy() 
    classes = result.boxes.cls.cpu().numpy() 
    
    class_names = result.names

    with open('output_predictions.txt', 'w') as f:
        for i in range(len(boxes)):
            class_id = int(classes[i])
            class_name = class_names[class_id]
            score = scores[i]
            x1, y1, x2, y2 = boxes[i]
            f.write(f'{class_id} {class_name} {score:.4f} {x1:.2f} {y1:.2f} {x2:.2f} {y2:.2f}\n')

print("Predictions saved to output_predictions.txt")


image 1/1 C:\Users\rajes\OneDrive\Desktop\DAYUMMMMMMMMM\CNN\images (1).jpg: 384x640 11 Cows, 3 Seals, 35.6ms
Speed: 2.5ms preprocess, 35.6ms inference, 61.6ms postprocess per image at shape (1, 3, 384, 640)
Predictions saved to output_predictions.txt


In [3]:
import requests
import json

# Ollama API endpoint
OLLAMA_ENDPOINT = "http://localhost:11434/api/generate"

# Read YOLO detection results
with open("output_predictions.txt", "r") as file:
    yolo_results = file.readlines()

# Format the YOLO results properly
formatted_results = []
for line in yolo_results:
    parts = line.strip().split()
    if len(parts) == 7:
        formatted_results.append(f"Detected {parts[1]} with {float(parts[2])*100:.1f}% confidence at coordinates ({parts[3]}, {parts[4]}, {parts[5]}, {parts[6]})")

# Read the prompt template
with open("prompt_template.txt", "r") as f:
    prompt_template = f.read()

# Replace the placeholder with formatted results
prompt = prompt_template.replace("%DETECTIONS%", chr(10).join(formatted_results))

# Request parameters
data = {
    "model": "imagedet2",
    "prompt": prompt,
    "stream": False
}

# Make the API request
response = requests.post(OLLAMA_ENDPOINT, json=data)
response_json = response.json()
print("\nImage Analysis:")
print(response_json['response'])

# Save the interpretation to a file
with open("interpretation_results.txt", "w") as f:
    f.write(response_json['response'])


Image Analysis:
Here are the YOLO detection outputs parsed and explained in the requested format:

**Detection 1: Cow**
{
  "detection": {
    "object_id": "1",
    "class": "Cow",
    "confidence_analysis": {
      "score": 74.7,
      "interpretation": "High confidence"
    },
    "spatial_info": {
      "position": "Top-left corner at (48.99, 111.94) and bottom-right corner at (101.41, 127.56)",
      "size": "Medium-sized object",
      "coordinates": {
        "top_left": [48.99, 111.94],
        "bottom_right": [101.41, 127.56]
      }
    },
    "natural_description": "A cow is located in the top-left corner of the image at coordinates (48.99, 111.94) and extends to the bottom-right corner at (101.41, 127.56). The object appears to be medium-sized."
  }
}

**Detection 2: Cow**
{
  "detection": {
    "object_id": "1",
    "class": "Cow",
    "confidence_analysis": {
      "score": 63.5,
      "interpretation": "Moderate confidence"
    },
    "spatial_info": {
      "position": 